In [120]:
from renix_py_api.renix import *
initialize(log_level=logging.INFO)
from renix_py_api.api_gen import *
from renix_py_api.core import EnumRelationDirection
from renix_py_api.rom_manager import *

# 占用测试仪接口
def create_ports(sys_entry, locations):
    renix_info('Create ports with locations:{}'.format(locations))
    port1 = Port(upper=sys_entry, location=locations[0])
    port2 = Port(upper=sys_entry, location=locations[1])
    assert port1.handle
    assert port2.handle
    bring_port_online_cmd = BringPortsOnlineCommand(portlist=[port1.handle, port2.handle])
    bring_port_online_cmd.execute()
    print(wait_port_online(port1))
    if not wait_port_online(port1):
        raise Exception('bring online port({}) failed'.format(port1.handle))
    if not wait_port_online(port2):
        raise Exception('bring online port({}) failed'.format(port2.handle))
    return port1, port2


# 创建带VLAN的数据量
def create_stream_vlan(port, packet_length=128):
    renix_info('port({}) create streams'.format(port_location))
    stream = StreamTemplate(upper=port)
    assert stream.handle
    create_stream_header_cmd = CreateHeaderCommand(stream.handle,
                                                   ['Ethernet.ethernetII', 'VLAN.vlan', 'IPv4.ipv4', 'UDP.udp'])
    create_stream_header_cmd.execute()
    if len(create_stream_header_cmd.HeaderNames) != 4:
        raise Exception('{} create EthernetII and IPv4 header failed'.format(stream.handle))
    stream.FixedLength = packet_length
    stream.get()
    return stream


# 创建不带VLAN的数据量
def create_stream(port, packet_length=128):
    renix_info('port({}) create streams'.format(port_location))
    stream = StreamTemplate(upper=port)
    assert stream.handle
    create_stream_header_cmd = CreateHeaderCommand(stream.handle, ['Ethernet.ethernetII', 'IPv4.ipv4', 'UDP.udp'])
    create_stream_header_cmd.execute()
    if len(create_stream_header_cmd.HeaderNames) != 3:
        raise Exception('{} create EthernetII and IPv4 header failed'.format(stream.handle))
    stream.FixedLength = packet_length
    stream.get()
    return stream


# 编辑数据流
def edit_stream(stream, parameter):
    update_header_cmd = UpdateHeaderCommand(Stream=stream.handle, Parameter=parameter)
    update_header_cmd.execute()
    stream.get()


# 等待测试仪端口上线
def wait_port_online(port, times=30):
    while times:
        if port.Online:
            return True
        else:
            times -= 1
            time.sleep(1)
    else:
        return False

renix python api init begin
log path: c:\users\luhairui\appdata\local\programs\python\python38\lib\site-packages\logs\ipykernel_launcher_Aug_11_20_09_15_04.log


[INFO] 2020-08-11 09:15:38,003  Receive Msg: listen_port 9001

[INFO] 2020-08-11 09:15:38,003  Receive Msg: listen_port 9001

[INFO] 2020-08-11 09:15:38,003  Receive Msg: listen_port 9001

[INFO] 2020-08-11 09:15:38,125  Renix api initialization finished
[INFO] 2020-08-11 09:15:38,125  Renix api initialization finished
[INFO] 2020-08-11 09:15:38,125  Renix api initialization finished


In [121]:
port_location  = ['//192.168.0.180/1/1', '//192.168.0.180/1/2']
stream_rate = "10"
stream_num = "100000"
download_capture_num = "100000"
olt_type = "FD1216S"
olt_mgt_ip = "192.168.5.82"
olt_username = "test111"
olt_password = "test123"
file_name = "FD314_504_704_EGX_RC_SFU_V1.0.9_200407_X000.tar"
tftp_server_ip = "192.168.5.221"
User_Vlan = "2000"
Vlan_list = "2000,4001"
Internet_Vlan = "4001"
Server_Vlan = "2000"
Ont_Srvprofile_ID = "9"
Ont_Lineprofile_ID = "4"
Ont_Igmpprofile_ID = "200"
Traffic_Profile_ID_inbound = "1"
Traffic_Profile_ID_outbound = "2"
ONU_MAC = 'E0:67:B3:88:88:88'
Dba_Profile_ID = "100"
FrameID = "0"
SlotID  = "0"
PonID = "0"
OnuID = "0"
Gemport_ID = "1"
Ont_Port_ID = "1"
SN = "52544B4711111125"
SN_PASSWORD  = "1234567890"
LOID = "zhongqi"
LOID_PASSWORD = "zhongqi"
Ping_test_addr = "www.baidu.com"
Network_car_name  = "ASIX AX88772C USB2.0 to Fast Ethernet Adapter"
pppoe_client = "宽带连接"
pppoe_name = "csb_1"
pppoe_password = "csb_1"

In [6]:
time.sleep(3)
reset_rom_cmd = ResetROMCommand()
reset_rom_cmd.execute()
# service-port配置完成后，需要等待一段时间流才能通
time.sleep(10)
sys_entry = get_sys_entry()

# 占用测试仪端口
port1, port2 = create_ports(sys_entry, port_location)

# 插卡测试仪接口当前的双工速率
print("===========================================================================")
print("当前端口1的协商速率为：%s" % ((port1.get_children('EthCopper')[0].LineSpeed)))
print("当前端口2的协商速率为：%s" % ((port2.get_children('EthCopper')[0].LineSpeed)))
print("===========================================================================")

# 配置测试仪Load Profiles模板(速率为10%，打流方式为突发方式，双向发送100000个报文)
stream_port_cfg_1 = port1.get_children(StreamPortConfig.cls_name())[0]
inter_frame_gap_cfg_1 = stream_port_cfg_1.get_children(InterFrameGapProfile.cls_name())[0]
inter_frame_gap_cfg_1.edit(Rate=int(stream_rate))
stream_port_cfg_2 = port2.get_children(StreamPortConfig.cls_name())[0]
inter_frame_gap_cfg_2 = stream_port_cfg_2.get_children(InterFrameGapProfile.cls_name())[0]
inter_frame_gap_cfg_2.edit(Rate=int(stream_rate))
stream_port_cfg_1.edit(TransmitMode=EnumTransmitMode.BURST)
stream_port_cfg_2.edit(TransmitMode=EnumTransmitMode.BURST)
stream_port_cfg_1.get()
stream_port_cfg_2.get()
Burst_transmit_config_1 = stream_port_cfg_1.get_children('BurstTransmitConfig')[0]
Burst_transmit_config_2 = stream_port_cfg_2.get_children('BurstTransmitConfig')[0]
Burst_transmit_config_1.edit(FramePerBurst=int(stream_num))
Burst_transmit_config_2.edit(FramePerBurst=int(stream_num))

# 创建数据流
stream1_2 = create_stream(port1)
stream2_1 = create_stream(port2)
edit_stream(stream1_2, 'ethernetII_1.sourceMacAdd=00:00:00:12:11:11 ethernetII_1.destMacAdd=00:00:00:21:22:22')
edit_stream(stream2_1, 'ethernetII_1.sourceMacAdd=00:00:00:21:22:22 ethernetII_1.destMacAdd=00:00:00:12:11:11')
# edit_stream(stream1_2,'vlan_1.id.XetModifier.StreamType=InterModifier vlan_1.id.XetModifier.Type=Increment vlan_1.id.XetModifier.Start=1001 vlan_1.id.XetModifier.Step=1 vlan_1.id.XetModifier.Count=5')
# edit_stream(stream2_1,'vlan_1.id.XetModifier.StreamType=InterModifier vlan_1.id.XetModifier.Type=Increment vlan_1.id.XetModifier.Start=1001 vlan_1.id.XetModifier.Step=1 vlan_1.id.XetModifier.Count=5')
edit_stream(stream1_2,
            'ipv4_1.destination.XetModifier.StreamType=InterModifier ipv4_1.destination.XetModifier.Type=Increment ipv4_1.destination.XetModifier.Start=192.168.0.222 ipv4_1.destination.XetModifier.Step=1 ipv4_1.destination.XetModifier.Count=5')
edit_stream(stream2_1,
            'ipv4_1.destination.XetModifier.StreamType=InterModifier ipv4_1.destination.XetModifier.Type=Increment ipv4_1.destination.XetModifier.Start=192.168.0.111 ipv4_1.destination.XetModifier.Step=1 ipv4_1.destination.XetModifier.Count=5')
edit_stream(stream1_2,
            'ipv4_1.source.XetModifier.StreamType=InterModifier ipv4_1.source.XetModifier.Type=Increment ipv4_1.source.XetModifier.Start=192.168.0.111 ipv4_1.source.XetModifier.Step=1 ipv4_1.source.XetModifier.Count=5')
edit_stream(stream2_1,
            'ipv4_1.source.XetModifier.StreamType=InterModifier ipv4_1.source.XetModifier.Type=Increment ipv4_1.source.XetModifier.Start=192.168.0.222 ipv4_1.source.XetModifier.Step=1 ipv4_1.source.XetModifier.Count=5')
stream1_2.set_relatives('RxPort', port2, EnumRelationDirection.TARGET)
stream2_1.set_relatives('RxPort', port1, EnumRelationDirection.TARGET)

# 配置测试仪查看测试结果的视图
resultView_create = CreateResultViewCommand(DataClassName=StreamBlockStats.cls_name())
resultView_create.execute()
resultView_create = ROMManager.get_object(resultView_create.ResultViewHandle)
subscribe_result_cmd = SubscribeResultCommand(ResultViewHandles=resultView_create.handle)
subscribe_result_cmd.execute()
# cap_conf_1 = port1.get_children('CaptureConfig')[0]
# cap_conf_2 = port2.get_children('CaptureConfig')[0]

# 开始测试仪抓包
# start_all_cap_cmd = StartAllCaptureCommand()
# start_all_cap_cmd.execute()
sys_entry.get()
page_result_view = sys_entry.get_children(PageResultView.cls_name())[0]

[INFO] 2020-08-10 20:25:10,684  Create ports with locations:['//192.168.0.180/1/1', '//192.168.0.180/1/2']
[INFO] 2020-08-10 20:25:10,684  Create ports with locations:['//192.168.0.180/1/1', '//192.168.0.180/1/2']
[INFO] 2020-08-10 20:25:19,523  EthCopper_3 has lower: EthCopperConfig_3
[INFO] 2020-08-10 20:25:19,523  EthCopper_3 has lower: EthCopperConfig_3
[INFO] 2020-08-10 20:25:19,542  EthCopper_4 has lower: EthCopperConfig_4
[INFO] 2020-08-10 20:25:19,542  EthCopper_4 has lower: EthCopperConfig_4
[INFO] 2020-08-10 20:25:19,559  StreamPortConfig_3 has lower: InterFrameGapProfile_3
[INFO] 2020-08-10 20:25:19,559  StreamPortConfig_3 has lower: InterFrameGapProfile_3
[INFO] 2020-08-10 20:25:19,580  StreamPortConfig_4 has lower: InterFrameGapProfile_4
[INFO] 2020-08-10 20:25:19,580  StreamPortConfig_4 has lower: InterFrameGapProfile_4
[INFO] 2020-08-10 20:25:19,649  port(['//192.168.0.180/1/1', '//192.168.0.180/1/2']) create streams


True
当前端口1的协商速率为：EnumLineSpeed.SPEED_1G
当前端口2的协商速率为：EnumLineSpeed.SPEED_1G


[INFO] 2020-08-10 20:25:19,649  port(['//192.168.0.180/1/1', '//192.168.0.180/1/2']) create streams
[INFO] 2020-08-10 20:25:19,850  port(['//192.168.0.180/1/1', '//192.168.0.180/1/2']) create streams
[INFO] 2020-08-10 20:25:19,850  port(['//192.168.0.180/1/1', '//192.168.0.180/1/2']) create streams
[INFO] 2020-08-10 20:25:21,170  PageResultView_1 has lower: ResultQuery_1
[INFO] 2020-08-10 20:25:21,170  PageResultView_1 has lower: ResultQuery_1


In [7]:
start_all_stream_cmd = StartAllStreamCommand()
start_all_stream_cmd.execute()
time.sleep(10)
result_query = page_result_view.get_children()[0]


[INFO] 2020-08-10 20:25:33,771  ResultQuery_1 has lower: StreamBlockStats_1
[INFO] 2020-08-10 20:25:33,771  ResultQuery_1 has lower: StreamBlockStats_1


In [8]:
print(result_query)
print(result_query.get_children())
print(result_query.get_children().__dict__)


AttributeError: 'list' object has no attribute '__dict__'

In [20]:
print(result_query.get_children()[0]._StreamBlockID)

IndexError: list index out of range

In [23]:
print(port2.__dict__)

{'_Location': '//192.168.0.180/1/2', '_Online': True, '_Name': 'Port_4', '_Enable': True, '_ROMTag': '', 'force_auto_sync': True, 'handle': 'Port_4', 'lower': [<renix_py_api.api_gen.SendPortLinkFault_Autogen.SendPortLinkFault object at 0x0000000009C78130>, <renix_py_api.api_gen.StreamPortConfig_Autogen.StreamPortConfig object at 0x0000000009C78340>, <renix_py_api.api_gen.ArpPortConfig_Autogen.ArpPortConfig object at 0x0000000009C78B80>, <renix_py_api.api_gen.TlsManager_Autogen.TlsManager object at 0x0000000009C78C40>, <renix_py_api.api_gen.CaptureConfig_Autogen.CaptureConfig object at 0x0000000009C78160>, <renix_py_api.api_gen.IgmpPortRateConfig_Autogen.IgmpPortRateConfig object at 0x0000000009C78D30>, <renix_py_api.api_gen.MldPortRateConfig_Autogen.MldPortRateConfig object at 0x0000000009C789A0>, <renix_py_api.api_gen.Dhcpv4PortConfig_Autogen.Dhcpv4PortConfig object at 0x0000000009C78940>, <renix_py_api.api_gen.Dhcpv4ServerPortConfig_Autogen.Dhcpv4ServerPortConfig object at 0x00000000

In [30]:
print(port1.get_children(SendPortLinkFault()))

[ERROR] 2020-08-10 20:37:18,397  b'_TCLERROR:isinstance() arg 2 must be a type or tuple of types'
[ERROR] 2020-08-10 20:37:18,397  b'_TCLERROR:isinstance() arg 2 must be a type or tuple of types'


RuntimeError: Port_3Failed get child nodes

In [35]:
sys_entry.__dict__

{'_Version': '3.1.2.629229',
 '_VersionType': 'alpha',
 '_TestCaseName': 'Untitled',
 '_EnableDebug': False,
 '_ProductNameSuffix': '',
 '_Name': 'SysEntry_1',
 '_Enable': True,
 '_ROMTag': '',
 'force_auto_sync': True,
 'handle': 'SysEntry_1',
 'lower': [<renix_py_api.api_gen.PlCommander_Autogen.PlCommander at 0x9c78a30>,
 'upper': None,
 'relatives': {}}

In [49]:
port1.get_children()[0].__dict__

[INFO] 2020-08-10 20:46:10,304  StreamPortConfig_3 has lower: InterFrameGapProfile_3 BurstTransmitConfig_1
[INFO] 2020-08-10 20:46:10,304  StreamPortConfig_3 has lower: InterFrameGapProfile_3 BurstTransmitConfig_1
[INFO] 2020-08-10 20:46:10,336  EthCopper_3 has lower: EthCopperConfig_3
[INFO] 2020-08-10 20:46:10,336  EthCopper_3 has lower: EthCopperConfig_3
[INFO] 2020-08-10 20:46:10,341  StreamTemplate_1 has lower: EthernetParameter_1 Ipv4Parameter_1 UdpParameter_1
[INFO] 2020-08-10 20:46:10,341  StreamTemplate_1 has lower: EthernetParameter_1 Ipv4Parameter_1 UdpParameter_1


{'_Mode': <EnumLinkFaultMode.DISCONTINUOUS: 2>,
 '_Duration': 1000,
 '_Running': False,
 '_Name': 'SendPortLinkFault_3',
 '_Enable': True,
 '_ROMTag': '',
 'force_auto_sync': True,
 'handle': 'SendPortLinkFault_3',
 'lower': [],
 'upper': <renix_py_api.api_gen.Port_Autogen.Port at 0x9c65430>,
 'relatives': {}}

In [108]:
port1.get_children()[0].__dict__

[INFO] 2020-08-10 20:58:24,434  StreamPortConfig_3 has lower: InterFrameGapProfile_3 BurstTransmitConfig_1
[INFO] 2020-08-10 20:58:24,434  StreamPortConfig_3 has lower: InterFrameGapProfile_3 BurstTransmitConfig_1
[INFO] 2020-08-10 20:58:24,468  EthCopper_3 has lower: EthCopperConfig_3
[INFO] 2020-08-10 20:58:24,468  EthCopper_3 has lower: EthCopperConfig_3
[INFO] 2020-08-10 20:58:24,474  StreamTemplate_1 has lower: EthernetParameter_1 Ipv4Parameter_1 UdpParameter_1
[INFO] 2020-08-10 20:58:24,474  StreamTemplate_1 has lower: EthernetParameter_1 Ipv4Parameter_1 UdpParameter_1


{'_Mode': <EnumLinkFaultMode.DISCONTINUOUS: 2>,
 '_Duration': 1000,
 '_Running': False,
 '_Name': 'SendPortLinkFault_3',
 '_Enable': True,
 '_ROMTag': '',
 'force_auto_sync': True,
 'handle': 'SendPortLinkFault_3',
 'lower': [],
 'upper': <renix_py_api.api_gen.Port_Autogen.Port at 0x9c65430>,
 'relatives': {}}

In [122]:
print(port1.get_children('EthCopper')[0]._LinkStatus)

[ERROR] 2020-08-11 09:15:54,771  b'_TCLERROR:object Port_3 does not exist'
[ERROR] 2020-08-11 09:15:54,771  b'_TCLERROR:object Port_3 does not exist'
[ERROR] 2020-08-11 09:15:54,771  b'_TCLERROR:object Port_3 does not exist'


RuntimeError: Port_3Failed get child nodes

In [62]:
port2.get_children()[0].__dict__

[INFO] 2020-08-10 20:48:56,985  StreamPortConfig_4 has lower: InterFrameGapProfile_4 BurstTransmitConfig_2
[INFO] 2020-08-10 20:48:56,985  StreamPortConfig_4 has lower: InterFrameGapProfile_4 BurstTransmitConfig_2
[INFO] 2020-08-10 20:48:57,022  EthCopper_4 has lower: EthCopperConfig_4
[INFO] 2020-08-10 20:48:57,022  EthCopper_4 has lower: EthCopperConfig_4
[INFO] 2020-08-10 20:48:57,028  StreamTemplate_2 has lower: EthernetParameter_2 Ipv4Parameter_2 UdpParameter_2
[INFO] 2020-08-10 20:48:57,028  StreamTemplate_2 has lower: EthernetParameter_2 Ipv4Parameter_2 UdpParameter_2


{'_Mode': <EnumLinkFaultMode.DISCONTINUOUS: 2>,
 '_Duration': 1000,
 '_Running': False,
 '_Name': 'SendPortLinkFault_4',
 '_Enable': True,
 '_ROMTag': '',
 'force_auto_sync': True,
 'handle': 'SendPortLinkFault_4',
 'lower': [],
 'upper': <renix_py_api.api_gen.Port_Autogen.Port at 0x50e6eb0>,
 'relatives': {}}

In [128]:
print(port2.get_children('EthCopper')[0]._LinkStatus._name_)

[INFO] 2020-08-11 09:17:18,803  EthCopper_2 has lower: EthCopperConfig_2
[INFO] 2020-08-11 09:17:18,803  EthCopper_2 has lower: EthCopperConfig_2
[INFO] 2020-08-11 09:17:18,803  EthCopper_2 has lower: EthCopperConfig_2


UP


In [110]:
sys_entry = get_sys_entry()

In [124]:
print(port1.get_children('EthCopper')[0].LineSpeed._name_)

[INFO] 2020-08-11 09:16:59,090  EthCopper_1 has lower: EthCopperConfig_1
[INFO] 2020-08-11 09:16:59,090  EthCopper_1 has lower: EthCopperConfig_1
[INFO] 2020-08-11 09:16:59,090  EthCopper_1 has lower: EthCopperConfig_1


SPEED_1G


In [123]:
time.sleep(3)
reset_rom_cmd = ResetROMCommand()
reset_rom_cmd.execute()
# service-port配置完成后，需要等待一段时间流才能通
time.sleep(10)
sys_entry = get_sys_entry()

# 占用测试仪端口
port1, port2 = create_ports(sys_entry, port_location)

[INFO] 2020-08-11 09:16:48,098  Create ports with locations:['//192.168.0.180/1/1', '//192.168.0.180/1/2']
[INFO] 2020-08-11 09:16:48,098  Create ports with locations:['//192.168.0.180/1/1', '//192.168.0.180/1/2']
[INFO] 2020-08-11 09:16:48,098  Create ports with locations:['//192.168.0.180/1/1', '//192.168.0.180/1/2']


True
